Agente Inteligente para Notas Fiscais - Google Colab
Este notebook implementa um agente inteligente capaz de ler um arquivo CSV com dados de Notas Fiscais e responder perguntas em linguagem natural sobre ele.


📌 Passo 1 — Instalar bibliotecas necessárias(OpenAI, langchain, pandas)


In [ ]:
!pip install openai langchain pandas
!pip install langchain_experimental
!pip install -U langchain-openai

📌 Passo 2 — Importar bibliotecas

In [32]:
import os
os.environ["OPENAI_API_KEY"] = "INSIRE A CHAVE AQUI"

In [33]:
import pandas as pd
from langchain import OpenAI
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent

In [34]:
import os
import pandas as pd
from langchain_experimental.agents import create_csv_agent
from langchain.llms import OpenAI

📌 Passo 3 — Conectar ao Google Drive

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


📌 Passo 4 — Descompactar arquivo ZIP

1.   Item da lista
2.   Item da lista


Agora, use o código para descompactar o arquivo ZIP. Lembre-se de substituir `'/content/drive/My Drive/caminho/para/seu_arquivo.zip'` pelo caminho real do seu arquivo no Google Drive e `'/content/drive/My Drive/caminho/para/extrair'` pelo diretório de destino desejado (também no seu Drive ou em outro local acessível).

In [36]:
import zipfile
import os

# Substitua pelo caminho completo do seu arquivo .zip no Google Drive
zip_file_path = '/content/drive/My Drive/Colab Notebooks/202401_NFs_Desafio_3.zip'

# Substitua pelo diretório onde você quer extrair os arquivos (pode ser no Drive ou em outro lugar)
extract_dir = '/content/drive/My Drive/Colab Notebooks/NotasFiscaisDescompactadas'

# Cria o diretório de destino se ele não existir
os.makedirs(extract_dir, exist_ok=True)

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Arquivo '{zip_file_path}' descompactado com sucesso em '{extract_dir}'.")
except FileNotFoundError:
    print(f"Erro: Arquivo '{zip_file_path}' não encontrado. Verifique o caminho.")
except zipfile.BadZipFile:
    print(f"Erro: O arquivo '{zip_file_path}' não é um arquivo ZIP válido.")
except Exception as e:
    print(f"Ocorreu um erro durante a descompactação: {e}")

Arquivo '/content/drive/My Drive/Colab Notebooks/202401_NFs_Desafio_3.zip' descompactado com sucesso em '/content/drive/My Drive/Colab Notebooks/NotasFiscaisDescompactadas'.


📌 Passo 5 — Ler os CSVs

In [ ]:
import pandas as pd

# Caminhos para os dois arquivos
cabecalho_path = '/content/drive/MyDrive/Colab Notebooks/NotasFiscaisDescompactadas/202401_NFs_Cabecalho.csv'
itens_path = '/content/drive/MyDrive/Colab Notebooks/NotasFiscaisDescompactadas/202401_NFs_Itens.csv'

# Lendo os CSVs
df_cabecalho = pd.read_csv(cabecalho_path)
df_itens = pd.read_csv(itens_path)

# Mostrando as primeiras linhas
print("📄 Cabeçalho:")
display(df_cabecalho.head())

print("📄 Itens:")
display(df_itens.head())

📌 Passo 6 — Criando Agente para o Cabeçalho
 🤖

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent

llm = ChatOpenAI(
  model="mistralai/mistral-7b-instruct",  # ou outro modelo suportado
  temperature=0,
  api_key="INSIRA A CHAVE AQUI",
  base_url="https://openrouter.ai/api/v1"
  )

agent_cabecalho = create_pandas_dataframe_agent(
llm,
df_cabecalho,
verbose=True,
allow_dangerous_code=True,
handle_parsing_errors=True
  )
agent_cabecalho.invoke("Quantas notas fiscais foram emitidas?")

📌 Passo 7 — Criando Agente para os Itens 🤖

In [ ]:
from langchain.prompts import PromptTemplate

# Prompt customizado
custom_prefix = """
Você é um agente especialista em análise de notas fiscais.
Sempre que perguntado sobre produtos mais vendidos, você deve procurar o valor máximo na coluna QUANTIDADE
e devolver a DESCRIÇÃO correspondente a essa linha no dataframe.
Responda em português e de forma objetiva.
"""

agent_itens = create_pandas_dataframe_agent(
llm,
df_itens,
verbose=True,
allow_dangerous_code=True,
prefix=custom_prefix
                                             )
agent_itens.invoke("Qual foi o produto mais vendido?")